In [21]:
from loguru import logger
import math
import subprocess
from pathlib import Path

import yaml

with open("../run.yaml", "r") as f:
    config_template = yaml.safe_load(f)

In [22]:
def write_slurm_script(work_dir: str | Path, config: dict, script_content: str) -> Path:
    """
    Generate a SLURM job script based on the provided configuration.

    Args:
        config (dict): Parsed YAML configuration.
        work_dir (str): Directory to write the SLURM job script.
        script_content (str): Bash commands to run in the SLURM job script.
    """

    work_dir = Path(work_dir)
    work_dir.mkdir(parents=True, exist_ok=True)

    with open(work_dir / "run.yaml", "w") as f:
        yaml.dump(config, f, sort_keys=False)

    scheduler = config["scheduler"]
    resources = scheduler["resources"]
    pre_run = scheduler.get("pre_run", "")
    post_run = scheduler.get("post_run", "")
    modules = scheduler.get("modules", []) or []

    # Define the SLURM script path
    script_path = work_dir / "job.slurm"

    # Write the SLURM script
    with open(script_path, "w") as f:
        f.write(f"""#!/bin/bash
#SBATCH --job-name={resources["job_name"]}
#SBATCH --output={resources["output"]}
#SBATCH --error={resources["error"]}
#SBATCH --time={resources["time"]}
#SBATCH --partition={resources["partition"]}
#SBATCH --nodes={resources["nodes"]}
#SBATCH --ntasks-per-node={resources["ntasks_per_node"]}

# Pre-run commands
{pre_run if pre_run else ""}

# Load required modules
""")

        for module in modules:
            f.write(f"module load {module}\n")

        f.write(f"""
{script_content}
""")

        f.write(f"\n# Post-run commands\n{post_run if post_run else ''}\n")

    logger.info(f"SLURM job script written to: {script_path}")
    return script_path

In [26]:
import itertools

ncpus_per_node = 64

NNODEs = [1, 2, 4, 8, 16]
# NCOREs = [1, 2, 4, 8, 16, 32, 64]
NCOREs = [2**i for i in range(int(math.log2(ncpus_per_node)) + 1)]
# KPARs = [1, 2, 4, 8, 16]  # <= KPOINTS = 2x2x4 = 16 in this case
KPARs = [2**i for i in range(int(math.log2(16)) + 1)]

for comb in itertools.product(NNODEs, NCOREs, KPARs):
    NNODE, NCORE, KPAR = comb
    NPAR = ncpus_per_node / KPAR / NCORE

    if not NPAR.is_integer():
        continue

    logger.info(f"NNODE: {NNODE}, NCORE: {NCORE}, KPAR: {KPAR}, NPAR: {NPAR}")
    logger.info(f"Total ranks (NNODE x NCORE x KPAR): {NNODE * NCORE * KPAR}")

    config = config_template.copy()

    config["job"]["vasp"]["INCAR"].update(
        {
            "NCORE": NCORE,
            "KPAR": KPAR,
            # "NPAR": int(NPAR),
        }
    )

    config["scheduler"]["resources"].update(
        {
            "nodes": NNODE,
            "ntasks_per_node": ncpus_per_node,
        }
    )

    work_dir = Path(f"{NNODE}-{NCORE}-{KPAR}")

    script_path = write_slurm_script(
        work_dir=work_dir,
        config=config,
        script_content="python ../workflow.py --config run.yaml",
    )

    subprocess.run(["sbatch", script_path.name], cwd=script_path.absolute().parent)
    logger.info(f"Submitted job {script_path}")
        

2025-01-17 14:27:58.740 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 1, KPAR: 1, NPAR: 64.0
2025-01-17 14:27:58.741 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 1
2025-01-17 14:27:58.745 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-1-1/job.slurm
2025-01-17 14:27:58.810 | INFO     | __main__:<module>:47 - Submitted job 1-1-1/job.slurm
2025-01-17 14:27:58.810 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 1, KPAR: 2, NPAR: 32.0
2025-01-17 14:27:58.811 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 2
2025-01-17 14:27:58.815 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-1-2/job.slurm
2025-01-17 14:27:58.861 | INFO     | __main__:<module>:47 - Submitted job 1-1-2/job.slurm
2025-01-17 14:27:58.863 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 1, KPAR: 4, NPAR: 16.0
2025-01-17 14:27:58.863 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR)

Submitted batch job 28409097
Submitted batch job 28409098
Submitted batch job 28409099


2025-01-17 14:27:59.012 | INFO     | __main__:<module>:47 - Submitted job 1-1-8/job.slurm
2025-01-17 14:27:59.013 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 1, KPAR: 16, NPAR: 4.0
2025-01-17 14:27:59.014 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 16
2025-01-17 14:27:59.018 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-1-16/job.slurm
2025-01-17 14:27:59.075 | INFO     | __main__:<module>:47 - Submitted job 1-1-16/job.slurm
2025-01-17 14:27:59.076 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 2, KPAR: 1, NPAR: 32.0
2025-01-17 14:27:59.076 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 2
2025-01-17 14:27:59.080 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-2-1/job.slurm
2025-01-17 14:27:59.110 | INFO     | __main__:<module>:47 - Submitted job 1-2-1/job.slurm
2025-01-17 14:27:59.111 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 2, KPAR: 2, NPAR: 16.0
2

Submitted batch job 28409100
Submitted batch job 28409101
Submitted batch job 28409102
Submitted batch job 28409103
Submitted batch job 28409104


2025-01-17 14:27:59.240 | INFO     | __main__:<module>:47 - Submitted job 1-2-8/job.slurm
2025-01-17 14:27:59.241 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 2, KPAR: 16, NPAR: 2.0
2025-01-17 14:27:59.241 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 32
2025-01-17 14:27:59.245 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-2-16/job.slurm
2025-01-17 14:27:59.282 | INFO     | __main__:<module>:47 - Submitted job 1-2-16/job.slurm
2025-01-17 14:27:59.283 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 4, KPAR: 1, NPAR: 16.0
2025-01-17 14:27:59.284 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 4
2025-01-17 14:27:59.287 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-4-1/job.slurm
2025-01-17 14:27:59.325 | INFO     | __main__:<module>:47 - Submitted job 1-4-1/job.slurm
2025-01-17 14:27:59.326 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 4, KPAR: 2, NPAR: 8.0
20

...

Submitted batch job 28409216
Submitted batch job 28409217


2025-01-17 14:28:07.199 | INFO     | __main__:<module>:47 - Submitted job 16-16-4/job.slurm
2025-01-17 14:28:07.200 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 32, KPAR: 1, NPAR: 2.0
2025-01-17 14:28:07.200 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 512
2025-01-17 14:28:07.204 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 16-32-1/job.slurm
2025-01-17 14:28:07.245 | INFO     | __main__:<module>:47 - Submitted job 16-32-1/job.slurm
2025-01-17 14:28:07.246 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 32, KPAR: 2, NPAR: 1.0
2025-01-17 14:28:07.247 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 1024
2025-01-17 14:28:07.250 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 16-32-2/job.slurm
2025-01-17 14:28:07.296 | INFO     | __main__:<module>:47 - Submitted job 16-32-2/job.slurm
2025-01-17 14:28:07.298 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 64, KPAR:

Submitted batch job 28409218
Submitted batch job 28409219
Submitted batch job 28409220


2025-01-17 14:28:07.402 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 16-64-1/job.slurm
2025-01-17 14:28:07.459 | INFO     | __main__:<module>:47 - Submitted job 16-64-1/job.slurm


Submitted batch job 28409221


In [27]:
from mp_api.client import MPRester

with MPRester() as mpr:
    docs = mpr.materials.summary.search(
        chemsys=["Pb-Ti-O", "Pb-Zr-O", "Pb-Ti-Zr-O"],
        spacegroup_symbol="P4mm",  # "Pm-3m", "P4mm" "R3mm"
        fields=["material_id", "symmetry", "structure"],
    )

structure = docs[0].structure

Retrieving SummaryDoc documents: 100%|██████████| 3/3 [00:00<00:00, 52428.80it/s]


In [29]:
with MPRester() as mpr:
    docs = mpr.materials.summary.search(
        chemsys=["Pb-Ti-O", "Pb-Zr-O", "Pb-Ti-Zr-O"],
        spacegroup_symbol="P4mm",  # "Pm-3m", "P4mm" "R3mm"
        fields=["material_id", "symmetry", "structure"],
    )

structure = docs[0].structure

structure = docs[1].structure * (3, 3, 2)
structure.to(filename=f"POSCAR.{structure.to_ase_atoms().get_chemical_formula()}")

Retrieving SummaryDoc documents: 100%|██████████| 3/3 [00:00<00:00, 79137.81it/s]
/ocean/projects/dmr110014p/ychiang4/.conda/envs/comm/lib/python3.12/site-packages/pymatgen/io/vasp/inputs.py:660: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


'Zr18 Ti18 Pb36 O108\n1.0\n  12.0827400000000011    0.0000000000000000    0.0000000000000000\n   0.0000000000000000   12.0827400000000011    0.0000000000000000\n   0.0000000000000000    0.0000000000000000   17.5231080000000006\nZr Ti Pb O\n18 18 36 108\ndirect\n   0.1666666666666667    0.1666666666666667    0.3819460000000000 Zr\n   0.1666666666666667    0.1666666666666667    0.8819460000000001 Zr\n   0.1666666666666667    0.5000000000000000    0.3819460000000000 Zr\n   0.1666666666666667    0.5000000000000000    0.8819460000000001 Zr\n   0.1666666666666667    0.8333333333333333    0.3819460000000000 Zr\n   0.1666666666666667    0.8333333333333333    0.8819460000000001 Zr\n   0.5000000000000000    0.1666666666666667    0.3819460000000000 Zr\n   0.5000000000000000    0.1666666666666667    0.8819460000000001 Zr\n   0.5000000000000000    0.5000000000000000    0.3819460000000000 Zr\n   0.5000000000000000    0.5000000000000000    0.8819460000000001 Zr\n   0.5000000000000000    0.83333333333